# Error Correcting Neural Network

### TL:DR

1. Mix each color using `pumpbot` and measure color values
    - Also mix [0.25, 0.25, 0.25, 0.25] on `pumpbot`
2. Set `true_coefficients`of `silicobot` to the individual `pumpbot` colors.
    - Also Mix [0.25, 0.25, 0.25, 0.25] on `silicobot` and measure difference to `pumpbot`
3. Do Bayesian Optimization in silico by changing weights $w$
    - Input: $w \cdot [0.25, 0.25, 0.25, 0.25]$ in silico
    - Output: $\mathrm{diff}(\text{weighted silicobot measured color}, \text{pumpbot measured color})$
4. Use best weight set and silicobot to create an ML that models the difference between `pumpbot` and `silicobot` using previous data from `pumpbot`:
    - Input: $w^* \cdot color$ in silico
    - Output: `pumpbot` mixed color - `silicobot` mixed color
    - This model can now take a silico mixed color and output the difference between this and the pumpbot color. This method requires that you have a bunch of `pumpbot` data
    - You can use Bayesian optimization to optimize number of hidden layers and hidden layer sizes of the NN model
5a. Generate data using error corrected `silicobot`:
    - 1000 random mixtures multiplied by best weights found in **5**.
    - Mixed colors are "measured" on the `silicobot`. Run the mixtures through the Error correcting ML model and add the resulting errors to the "measured" colors.
5b. Reverse the data and train a new model:
    - The measured colors (including error correction) are now the input to an new ML model.
    - The output is the mixture values.
6. Apply the reverse model:
    1. Measure random target color.
    2. Input measured RGB value in NN model
    3. Use NN model output mixture to mix color.
    4. Measure the newly mixed color and compare to target color.

### 0. Import packages

Start by importing the required packages for PumpController, SilicoPumpController, Odyssey and more...

In [ ]:
# PumpController and SilicoPumpController
from pump_controller import PumpController, get_serial_port, list_serial_ports
from pump_controller import SilicoPumpController
from pump_controller import visualize_rgb, visualize_candidates
from pump_controller import read_logfile

# Odyssey
from odyssey.mission import Mission # Mission
from odyssey.navigators import SingleGP_Navigator # Navigator
from odyssey.navigators.sampler_navigators import Sobol_Navigator # Sampler
from odyssey.navigators import ExpectedImprovement # Acquisition
from odyssey.objective import Objective # Objective

# Other Packages
import torch
import numpy as np
import pandas as pd  
import matplotlib.pyplot as plt  
from IPython import display
from warnings import catch_warnings, simplefilter

Define the `color_difference` function.

In [ ]:
# Difference between mixed and target colors:
def color_difference(mixed_color, target_color):

    mixed_color = np.array(mixed_color)
    target_color = np.array(target_color)
    # Calculate the sum of root mean squared differences between mixed color and target color
    rmse = np.sqrt(np.mean((mixed_color - target_color)**2, axis=-1))
    return np.sum(rmse)

### 1. Mix each individual color using pumpbot

Mix pure red, green, blue and yellow on the pump bot and plot and store these colors.

In [ ]:
pumpbot = PumpController(ser_port = get_serial_port(), cell_volume = 20.0, drain_time = 20.0, config_file = 'config.json')

In [ ]:
pumpbot_mixed_colors = []
for i in range(4):
    color_to_mix = [0,0,0,0]
    color_to_mix[i] = 1

    mixed_color = pumpbot.mix_color(color_to_mix)
    
    pumpbot_mixed_colors.append(mixed_color)

# Plot the colors
fig, axs = plt.subplots(1, 4, figsize=(12, 3))  # Create a 1x4 grid of subplots

for i, color in enumerate(pumpbot_mixed_colors):
    # Reshape the color to a 1x1x3 array and normalize to [0, 1]
    color = color.reshape(1, 1, 3) / 255.0

    axs[i].imshow(color)  # Display the color
    axs[i].axis('off')  # Hide the axes

plt.show()

Now, mix equal amounts of each color on pumpbot and visualize this color. We will use the `equal_color_mixture` variable multiple times during the course of this notebook.

In [ ]:
equal_color_mixture = [0.25,0.25,0.25,0.25]
pumpbot_equal_color = pumpbot.mix_color(equal_color_mixture)

visualize_rgb(mixture = equal_color_mixture, 
                  rgb = pumpbot_equal_color,
                  pump_controller = pumpbot,
                  target = None)

print(pumpbot_equal_color)

### 2. Create a silicobot using the colors obtained from pumpbot

Initialize the silicobot, but with the newly obtained `pumpbot_mixed_colors` as the `true_coefficients` of the silicobot

In [ ]:
silicobot = SilicoPumpController(noise_std = 0, true_coefficients = pumpbot_mixed_colors)

Mix equal amounts of each color on the silicobot. You can use the `equal_color_mixture` variable here.

In [ ]:
silicobot_equal_color = silicobot.mix_color(equal_color_mixture)

Visualize the mixed color and compare it to the color obtained on the pumpbot. This is an indicator of the difference between the pumpbot and the silicobot.

In [ ]:
score = color_difference(silicobot_equal_color, pumpbot_equal_color)

visualize_rgb(mixture = equal_color_mixture, 
                  rgb = silicobot_equal_color,
                  pump_controller = silicobot,
                  target = pumpbot_equal_color,
                  score = score)

### 3. Weights Bayesian Optimization

We will now optimize the weights of the silicobot. The objective will be to optimize the difference between the silicobot and pumpbot, when mixing the same color (equal amounts of each color). The parameters we will be tuning are the weights $w = [w_1, w_2, w_3, w_4]^T$, so that the color mixture mixed by the silicobot is $w \cdot [0.25, 0.25, 0.25, 0.25]$.

We can do this using Bayesian Optimization through Odyssey.

In [ ]:
def find_weights(weights):
    weighted_silicobot_equal_color = silicobot.mix_color(weights * torch.tensor([0.25, 0.25, 0.25, 0.25]))
    score = color_difference(weighted_silicobot_equal_color, pumpbot_equal_color)
    return score

objective = Objective(find_weights)

In [ ]:
goals = ['descend']

min = 1e-5

param_space = ([min, 1.0], [min, 1.0], [min, 1.0], [min, 1.0])

# Define Mission
mission = Mission(name = 'Silico Error Correcting',
                  funcs = [objective],
                  maneuvers = goals,
                  envelope = param_space)

In [ ]:
num_init_design = 5
navigator = SingleGP_Navigator(mission = mission,
                               num_init_design = num_init_design,
                               init_method = Sobol_Navigator(mission = mission),
                               input_scaling = False,
                               data_standardization = False,
                               display_always_max = False,
                               acq_function_type = ExpectedImprovement,
                               acq_function_params = {'best_f': 0.0})

In [ ]:
num_iter = 30
while len(mission.train_X) - num_init_design < num_iter:

    with catch_warnings() as w:
        simplefilter('ignore')
        
        trajectory = navigator.trajectory()
        observation = navigator.probe(trajectory, init = False)

        navigator.relay(trajectory, observation)
        navigator.upgrade()

Plot the different iterations and the score, and find the weights that allowed for the lowest difference (score).

In [ ]:
inputs = mission.display_X
scores = mission.display_Y

fig = plt.figure(figsize=(10, 5))

plt.scatter(range(len(scores)), scores)
plt.show()

Find the best weights (the ones where the score is lowest).

In [ ]:
best_idx = scores.argmin().item()
best_weights = inputs[best_idx]
best_score = scores[best_idx]

print(best_weights)

Mix equal amounts of this color, but this time weighted by the best weights found using the previous bayesian optimization.

In [ ]:
best_weighted_mixed_color = silicobot.mix_color(best_weights * torch.tensor([0.25, 0.25, 0.25, 0.25]))

Visualize this mixed color and compare to the equal mixtures color achieved on the pumpbot. Hopefully the difference is lower after using the optimized weights! 

In [ ]:
score = color_difference(best_weighted_mixed_color, pumpbot_equal_color)

print(f'Weights: {torch.round(best_weights, decimals = 3)}')
visualize_rgb(mixture = equal_color_mixture, 
                  rgb = best_weighted_mixed_color,
                  pump_controller = silicobot,
                  target = pumpbot_equal_color,
                  score = score)

### 4. Accumulate Old Data or Generate New Data for Error Correction NN Model

In this step, we are creating a neural network model, that can predict the error between our silicobot and the pumpcontroller to get the error even further down. We do this by combining all the data that we have from previous runs of the pumpbot, and using those mixtures as mixtures for the silicobot (weighted by the weights found earlier), and then calculating the error between the color that the pumpbot previously achieved and what the silicobot outputs. Using this data, we can train a neural network model. The more data that we have from the pumpbot, the more accurate this model will be.

#### 4a. Load Old Data

Get all your files in one folder and load them  all automatically, or manually make a list of your previous data.

In [ ]:
from glob import glob
files = glob('student_data/*.csv') # Get all files in a directory
# files = ['datafile1.csv', 'datafile2.csv', 'datafile3.csv'] # Or specify the files manually

Concatenate the dataframes, remove nan rows and restructure the dataframe.

In [ ]:
import ast

for i in range(len(files)):
    if i == 0:
        df = pd.read_csv(files[i])
    else:
        df = pd.concat([df, pd.read_csv(files[i])])

df = df.dropna() # Remove any nan rows
df = df.reset_index(drop=True) # Reset index

for column in df.columns:
        df[column] = df[column].apply(ast.literal_eval)

# Convert the lists of strings into lists of floats
for column in df.columns:
    df[column] = df[column].apply(lambda x: np.array([float(i) for i in x]))


The input data for this model is now generated by using the mixtures from the pumpbot, and weighted by the best weights found for the silicobot, and normalizing these values.

The output data is the error between the silicobot and the pumpbot for these mixtures. This can be calculated by running the silicobot on the mixture and subtracting this color from the pumpbot color made by using the same mixture.



In [ ]:
# If data is loaded 

X = torch.tensor(df.mixture) # Get the input data from pumpbot results
X = best_weights * X # Scale the points with the best weights
X /= X.sum(dim=1, keepdim=True) # Normalize the points


Y = torch.tensor(df.measurement)
for i in range(len(Y)):
    error = Y[i] - torch.tensor(silicobot.mix_color(X[i]))
    Y[i] = error

#### 4b. ... or Create New Data

If you do not have previous data (or want to add more data), you can generate it here as well, but this could take a while as it involves running the pumpbot.

In [ ]:
# If data is generated 

X = torch.rand(60, 4) # Generate random points
X = best_weights * X # Scale the points with the best weights
X /= X.sum(dim=1, keepdim=True) # Normalize the points

Y = []
for i in range(len(X)):
    error = pumpbot.mix_color(X[i]) - silicobot.mix_color(X[i])
    Y.append(error)

Y = torch.stack(Y) 

Using functions from pytorch, we can create a train-test split (here an 80-20 split), and create dataloaders (not to be confused by the Odyssey DataLoader) for the train and test data. These dataloaders get the data ready for training.

In [ ]:
from torch.utils.data import DataLoader, random_split, TensorDataset

# Create train, dependent test data
dataset = TensorDataset(X, Y)

total_size = len(dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=train_size, shuffle=True) # Use all data
test_loader = DataLoader(test_dataset, batch_size=test_size, shuffle=False) # Use all data

#### 4c. Create the Error Correction ML Model and Train Hyperparameters using Bayesian Optimization with Odyssey

We start by creating a general class for an Multi-Layer-Perceptron Regressor. This class takes an input size (in this case 4 nodes - one for each mixture color RGBY), an output size (in this case 3 nodes - one for each measurement color RGB) and a list of hidden layers. As you know, a neural network has a certain number of hidden layers each with a certain amount of nodes, and these numbers can be passed using the `hidden_layers` argument as a list. For example, creating 3 hidden layers with 2,3 and 4 nodes, respectively:

$$\texttt{hidden\_layers} = [2, 3, 4]$$

In [ ]:
import torch.nn as nn
import torch.optim as optim

class MLPRegressor(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, dropout_prob=0.5):
        super(MLPRegressor, self).__init__()
        self.layers = nn.ModuleList()
        last_size = input_size
        for size in hidden_layers:
            if size > 0:  # Only add the layer if it has more than 0 nodes
                self.layers.append(nn.Linear(last_size, size))
                self.layers.append(nn.BatchNorm1d(size))
                self.layers.append(nn.ReLU())
                self.layers.append(nn.Dropout(dropout_prob))
                last_size = size
        self.layers.append(nn.Linear(last_size, output_size))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

The number of hidden layers and nodes in each of them can be a mystery sometimes, but luckily we have a tool to help us search these kinds of spaces - Bayesian Optimization!

We create an objective function for bayesian optimization in odyssey. This function creates a MLPRegressor model with 4 input layers and 3 output layers and with a given number of hidden layers with given amounts of nodes in each hidden layer. We set the model in train mode, and train it for 100 epochs. After training, the model with the given hidden layers is evaluated on the test data using the `test_loader`. The loss function used for model training and testing is the root mean squared error loss (rMSE loss ). When testing, the average RMSE loss over all tested batches is returned as the output of this function.

To simplify, this function takes a list of hidden layers and their nodes as an input, and outputs the results of the model after training and testing. Using this function, we can optimize the number of hidden layers and nodes!

Note: Odyssey optimizes assuming that the input data is continuous. Obviously, the number of hidden layers and their nodes are all discrete values. The workaround for this is to simply convert the suggested number of continuous hidden layer configurations to integer values.

In [ ]:
def train_and_evaluate_model(hidden_layers):
    # Round the number of nodes in each hidden layer to the nearest integer
    hidden_layers = hidden_layers.to(torch.int64).squeeze()

    # Create an instance of the MLP regressor
    model = MLPRegressor(4, 3, hidden_layers)

    model.train()
    # Train the model
    loss_fn = nn.MSELoss() # Mean squared error loss

    optimizer = optim.Adam(model.parameters())
    n_epochs = 100
    for epoch in range(n_epochs):
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            loss = torch.sqrt(loss) # turn mse to rmse loss for each batch
            loss.backward()
            optimizer.step()

    model.eval()
    # Evaluate the model
    total_loss = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            total_loss += torch.sqrt(loss_fn(outputs, targets)) # turn mse to rmse loss for each batch

    return total_loss / len(test_loader) # average rmse over all batches


In [ ]:
objective = Objective(train_and_evaluate_model)

In this example, a maximum of 5 hidden layers is created, with each hidden layers having 0 to 20 nodes. If a layer has 0 nodes, it is obviously skipped. Now we can use odyssey to optimize the number of hidden layers

In [ ]:
goals = ['descend']

min_hidden_layer_nodes = 0.0
max_hidden_layer_nodes = 20.0
max_n_hidden_layers = 5

param_space = [[min_hidden_layer_nodes, max_hidden_layer_nodes]] * max_n_hidden_layers

# Define Mission
mission = Mission(name = 'Mix2RGB MLP Tuning',
                  funcs = [objective],
                  maneuvers = goals,
                  envelope = param_space)

In [ ]:
num_init_design = 5
navigator = SingleGP_Navigator(mission = mission,
                               num_init_design = num_init_design,
                               init_method = Sobol_Navigator(mission = mission),
                               input_scaling = False,
                               data_standardization = False,
                               display_always_max = False,
                               acq_function_type = ExpectedImprovement,
                               acq_function_params = {'best_f': 0.0})

In [ ]:
num_iter = 50
while len(mission.train_X) - num_init_design < num_iter:

    with catch_warnings() as w:
        simplefilter('ignore')
        
        trajectory = navigator.trajectory()
        observation = navigator.probe(trajectory, init = False)

        navigator.relay(trajectory, observation)
        navigator.upgrade()

We can figure out which configuration of hidden layer sizes resulted in the best results:

In [ ]:
best_idx = mission.display_Y.argmin()
best_params = mission.display_X[best_idx]
best_metric = mission.display_Y[best_idx]

In [ ]:
print(best_params.to(torch.int64))
print(best_metric)

We can now use the best achieved hidden layer configuration and retrain the model, this time for more epochs (more time consuming, but possibly more accurate).

In [ ]:
# Retrain the best model
forward_model = MLPRegressor(4, 3, best_params.to(torch.int64))

forward_model.train()
# Train the model
loss_fn = nn.MSELoss() # Mean squared error loss

optimizer = optim.Adam(forward_model.parameters())
n_epochs = 500
for epoch in range(n_epochs):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = forward_model(inputs)
        loss = loss_fn(outputs, targets)
        loss = torch.sqrt(loss) # turn mse to rmse loss for each batch
        loss.backward()
        optimizer.step()

We can now use this model to predict the error correction between the silicobot and the pumpbot for the equal amounts case.

In [ ]:
inputs = best_weights * torch.tensor([[0.25, 0.25, 0.25, 0.25]])

forward_model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Temporarily turn off gradient computation
    error_correction = forward_model(inputs)  # Make predictions

We run the silicobot on the (weighted) equal amounts and add the previously found error correction. Now we can calculate the difference between this corrected color and the original pumpbot color in the equal amounts case.

In [ ]:
# Mix a color with silicobot and error correct using the model
# Compare the color with the pumpbot
corrected_equal_color = silicobot.mix_color(best_weights * torch.tensor([0.25, 0.25, 0.25, 0.25])) + error_correction.squeeze().tolist()
score = color_difference(corrected_equal_color,pumpbot_equal_color)

Visualize this color!

In [ ]:
visualize_rgb(mixture = equal_color_mixture,
              rgb=corrected_equal_color,
              pump_controller=silicobot,
              target=pumpbot_equal_color,
              score=score)

Phew! That was a lot of work! Hopefully the difference between this error-corrected model and the original pumpbot color is very low!

### 5. Create a Reverse NN model to predict color mixture from color measurement

Hopefully we now have a silicobot, that with error correction is as close of a digital twin as possible to the original pumpbot. We can now generate a bunch of random mixtures and then use our digital twin to predict the colors that the pumpbot might output. 

In the end, we want to be able to take a color measurement (RGB) of a target color, and figure out what the mixture to reproduce that color should be. After having generated the random mixtures and the resulting (error corrected) color from our digital twin, we can use the mixtures as an output and the color values as an input to train another NN model. The hope is that we can then take any color measurement, and this model will give us the mixture required to reproduce it! We will call this model the reverse model, to distinguish from the error correction model (forward model).

#### 5a. Generate Data

Start by generating a lot of random mixtures, weighting them and normalizing. These values will be the outputs of our NN model.

In [ ]:
n_points = 1000
Y_reverse = best_weights * torch.rand(n_points,4)
Y_reverse /= Y_reverse.sum(dim=1, keepdim=True)

We calculate the color using the silicobot and the error correction from the previous NN model. These values will be the inputs of our NN model.

In [ ]:
X_reverse = []
with torch.no_grad():
    for i in range(len(Y_reverse)):
        x = torch.tensor(silicobot.mix_color(Y_reverse[i])) + forward_model(Y_reverse[i].unsqueeze(0)).squeeze()
        X_reverse.append(x)

X_reverse = torch.stack(X_reverse)

Like before, we create a train and test dataset, along with their trainloaders.

In [ ]:
# Create train, and test data for reverse model
reverse_dataset = TensorDataset(X_reverse, Y_reverse)

reverse_total_size = len(reverse_dataset)
reverse_train_size = int(0.8 * reverse_total_size)
reverse_test_size = reverse_total_size - reverse_train_size

reverse_train_dataset, reverse_test_dataset = random_split(reverse_dataset, [reverse_train_size, reverse_test_size])

# Dataloaders
reverse_train_loader = DataLoader(reverse_train_dataset, batch_size=reverse_train_size, shuffle=True) # Use all data
reverse_test_loader = DataLoader(reverse_test_dataset, batch_size=reverse_test_size, shuffle=False) # Use all data

#### 5b. Train the reverse model

Create the objective function to optimize the hidden layers as with the forward model. This time, the difference is that the MLPRegressor input layer has three nodes, one for each color in the measurement, and the output layer has four nodes, one for each mixture color. The model with the given hidden layer configuration is trained for 100 epochs the model is evaluated on the data.

In [ ]:
def train_and_evaluate_reverse_model(hidden_layers):
    # Round the number of nodes in each hidden layer to the nearest integer
    hidden_layers = hidden_layers.to(torch.int64).squeeze()

    # Create an instance of the MLP regressor
    model = MLPRegressor(3, 4, hidden_layers)

    model.train()
    # Train the model
    loss_fn = nn.MSELoss() # Mean squared error loss

    optimizer = optim.Adam(model.parameters())
    n_epochs = 100
    for epoch in range(n_epochs):
        for inputs, targets in reverse_train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            loss = torch.sqrt(loss) # turn mse to rmse loss for each batch
            loss.backward()
            optimizer.step()

    model.eval()
    # Evaluate the model
    total_loss = 0
    with torch.no_grad():
        for inputs, targets in reverse_test_loader:
            outputs = model(inputs)
            total_loss += torch.sqrt(loss_fn(outputs, targets)) # turn mse to rmse loss for each batch

    return total_loss / len(reverse_test_loader) # average rmse over all batches


In [ ]:
reverse_objective = Objective(train_and_evaluate_reverse_model)

Again, the hidden layer configuration is optimized using odyssey, with 5 maximum hidden layers, with each layer having 0 to 20 nodes.

In [ ]:
goals = ['descend']

min_hidden_layer_nodes = 0.0
max_hidden_layer_nodes = 20.0
max_n_hidden_layers = 5

param_space = [[min_hidden_layer_nodes, max_hidden_layer_nodes]] * max_n_hidden_layers

# Define Mission
mission = Mission(name = 'RGB2Mix MLP Tuning',
                  funcs = [reverse_objective],
                  maneuvers = goals,
                  envelope = param_space)

In [ ]:
num_init_design = 5
navigator = SingleGP_Navigator(mission = mission,
                               num_init_design = num_init_design,
                               init_method = Sobol_Navigator(mission = mission),
                               input_scaling = False,
                               data_standardization = False,
                               display_always_max = False,
                               acq_function_type = ExpectedImprovement,
                               acq_function_params = {'best_f': 0.0})

In [ ]:
num_iter = 15
while len(mission.train_X) - num_init_design < num_iter:

    with catch_warnings() as w:
        simplefilter('ignore')
        
        trajectory = navigator.trajectory()
        observation = navigator.probe(trajectory, init = False)

        navigator.relay(trajectory, observation)
        navigator.upgrade()

The best hidden layers configuration can now be found.

In [ ]:
best_idx = mission.display_Y.argmin()
best_params = mission.display_X[best_idx]
best_metric = mission.display_Y[best_idx]

In [ ]:
print(best_params.to(torch.int64))
print(best_metric)

A model is created with the best achieved hidden layer configuration and trained for more epochs.

In [ ]:
# Retrain the best model
reverse_model = MLPRegressor(3, 4, best_params.to(torch.int64))

reverse_model.train()
# Train the model
loss_fn = nn.MSELoss() # Mean squared error loss

optimizer = optim.Adam(reverse_model.parameters())
n_epochs = 500
for epoch in range(n_epochs):
    for inputs, targets in reverse_train_loader:
        optimizer.zero_grad()
        outputs = reverse_model(inputs)
        loss = loss_fn(outputs, targets)
        loss = torch.sqrt(loss) # turn mse to rmse loss for each batch
        loss.backward()
        optimizer.step()

### 6. Apply the reverse model on the pumpbot

Start by adding a target color to the test_cell and measure the color.

In [ ]:
measured_color = pumpbot.measure()
print(measured_color)

Get the proposed color mixture of this color using the reverse model.

In [ ]:
reverse_model.eval()
with torch.no_grad():
    color_to_mix = reverse_model(measured_color)
    print(color_to_mix)

Use the pumpbot to mix the proposed mixture.

In [ ]:
mixed_color = pumpbot.mix_color(color_to_mix)
print(mixed_color)

Compare the newly mixed color to the target color measurement. Hopefully the colors are identical both visually and in terms of the score!

In [ ]:
visualize_rgb(mixture = color_to_mix.squeeze().tolist(),
              rgb = mixed_color,
              pump_controller=pumpbot,
              target=measured_color.squeeze().tolist(),
              score=color_difference(mixed_color, measured_color))

A lot of work went to getting to this point, and even if you did not get a color that matched the original color, don't worry! You have learned a lot about digital twins, neural network models and using bayesian optimization for tuning hyperparameters and so much more!